# Summary of time taken and brier scores for jaxlogit, xlogit, and biogeme
Where the estimation is using draws = 600 (suboptimal but highest without running out of memory in biogeme), and training and test data is separated.

| | jaxlogit | xlogit | biogeme |
|---|---|---|---|
|Making Model | 37.7s | 16.9s | 4:15 |
|Estimating | 1.6s | 0.0s | 15.4s |
|Brier Score | 0.6345 | 0.6345 | 0.6345 |

In [4]:
import os
os.chdir("/home/evelyn/projects_shared/jaxlogit/examples")

# Setup

In [2]:
import pandas as pd
import numpy as np
import jax
import pathlib
import xlogit
import sklearn

from jaxlogit.mixed_logit import MixedLogit, ConfigData
from jaxlogit.utils import wide_to_long

#  64bit precision
jax.config.update("jax_enable_x64", True)

# Get the full electricity dataset

Use for jaxlogit and xlogit. Adjustusting n_draws can improve accuracy, but Biogeme cannot handle 700 or more draws with this data set.

In [5]:
df = pd.read_csv(pathlib.Path.cwd() / "electricity_long.csv")
varnames = ['pf', 'cl', 'loc', 'wk', 'tod', 'seas']
n_draws = 600

Reshape the data so it can be passed to test_train_split in a wide format. Additionally, xlogit and jaxlogit require long format while biogeme requires a wide format.

In [6]:
df_long = pd.read_csv(pathlib.Path.cwd() / "electricity_long.csv")
choice_df = df_long.loc[df_long['choice'] == 1, ['id', 'chid', 'alt']]
choice_df = choice_df.rename(columns={'alt': 'choice'})
df_wide = df_long.pivot(index=['id', 'chid'], columns='alt', values=varnames)
df_wide.columns = [f'{var}_{alt}' for var, alt in df_wide.columns]
df_wide = df_wide.reset_index()
df = df_wide.merge(
    choice_df,
    on=['id', 'chid'],
    how='inner',
    validate='one_to_one'
)

df_wide_train, df_wide_test = sklearn.model_selection.train_test_split(df, train_size=0.8)
df_train = wide_to_long(df_wide_train, "chid", [1,2,3,4], "alt", varying=varnames, panels=True)
df_train = df_train.sort_values(['chid', 'alt'])
df_test = wide_to_long(df_wide_test, "chid", [1,2,3,4], "alt", varying=varnames, panels=True)
df_test = df_test.sort_values(['chid', 'alt'])

In [7]:
X_train = df_train[varnames]
y_train = df_train['choice']

ids_train = df_train['chid']
alts_train = df_train['alt']
panels_train = df_train['id']

X_test = df_test[varnames]
y_test = df_test['choice']

ids_test = df_test['chid']
alts_test = df_test['alt']
panels_test = df_test['id']

In [8]:
randvars = {'pf': 'n', 'cl': 'n', 'loc': 'n', 'wk': 'n', 'tod': 'n', 'seas': 'n'}

model_jax = MixedLogit()
model_x = xlogit.MixedLogit()

config = ConfigData(
    panels=panels_train,
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)
init_coeff = None

Make the model in jaxlogit

In [9]:
model_jax.fit(
    X=X_train,
    y=y_train,
    varnames=varnames,
    ids=ids_train,
    alts=alts_train,
    randvars=randvars,
    config=config
)
display(model_jax.summary())
init_coeff_j = model_jax.coeff_

    Message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
    Iterations: 93
    Function evaluations: 112
Estimation time= 38.1 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
pf                     -1.0443131     1.0000000    -1.0443131         0.296    
cl                     -0.2374644     1.0000000    -0.2374644         0.812    
loc                     2.4178297     1.0000000     2.4178297        0.0157 *  
wk                      1.6528008     1.0000000     1.6528008        0.0985 .  
tod                   -10.0625781     1.0000000   -10.0625781       1.7e-23 ***
seas                  -10.1197586     1.0000000   -10.1197586      9.66e-24 ***
sd.pf                  -1.3244037     1.0000000    -1.3244037         0.185    
sd.cl                  -0.6750641     1.0000000    -0.6750641      

None

Make the model in xlogit

In [10]:
model_x.fit(
    X=X_train,
    y=y_train,
    varnames=varnames,
    ids=ids_train,
    alts=alts_train,
    randvars=randvars,
    panels=panels_train,
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)
display(model_x.summary())
init_coeff_x = model_x.coeff_

Optimization terminated successfully.
    Message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
    Iterations: 69
    Function evaluations: 79
Estimation time= 19.1 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
pf                     -1.0377335     1.0000000    -1.0377335         0.299    
cl                     -0.2314231     1.0000000    -0.2314231         0.817    
loc                     2.3429248     1.0000000     2.3429248        0.0192 *  
wk                      1.6208623     1.0000000     1.6208623         0.105    
tod                    -9.8530732     1.0000000    -9.8530732      1.32e-22 ***
seas                  -10.0301808     1.0000000   -10.0301808      2.34e-23 ***
sd.pf                  -0.2378622     1.0000000    -0.2378622         0.812    
sd.cl                   0.3933

None

Predict from the model using jaxlogit

In [11]:
model = model_jax 
config = ConfigData(
    panels=panels_test,
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)
config.init_coeff = init_coeff_j

In [12]:
prob_jj = model.predict(X_test, varnames, alts_test, ids_test, randvars, config)

Predict from the model using xlogit

In [13]:
model = model_x
_, prob_xx = model.predict(X_test, varnames, alts_test, ids_test, isvars=None, panels=panels_test, n_draws=n_draws, return_proba=True)

# Biogeme

In [22]:
import biogeme.biogeme_logging as blog
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Draws, log, MonteCarlo, PanelLikelihoodTrajectory
import biogeme.database as db
from biogeme.expressions import Variable
from biogeme.results_processing import get_pandas_estimated_parameters
import numpy as np
import sklearn

logger = blog.get_screen_logger()
logger.setLevel(blog.INFO)

In [15]:
database_train = db.Database('electricity', df_wide_train)
database_train.panel('id')
database_test = db.Database('electricity', df_wide_test)

Create variables and expression for utility of each alternative (V).

In [16]:
X = {
    name: {
        j: Variable(f"{name}_{j}")
        for j in [1,2,3,4]
    }
    for name in varnames
}

alt_1 = Beta('alt_1', 0, None, None, 0)
alt_2 = Beta('alt_2', 0, None, None, 0)
alt_3 = Beta('alt_3', 0, None, None, 0)
alt_4 = Beta('alt_4', 0, None, None, 1)

pf_mean = Beta('pf_mean', 0, None, None, 0)
pf_sd = Beta('pf_sd', 1, None, None, 0)
cl_mean = Beta('cl_mean', 0, None, None, 0)
cl_sd = Beta('cl_sd', 1, None, None, 0)
loc_mean = Beta('loc_mean', 0, None, None, 0)
loc_sd = Beta('loc_sd', 1, None, None, 0)
wk_mean = Beta('wk_mean', 0, None, None, 0)
wk_sd = Beta('wk_sd', 1, None, None, 0)
tod_mean = Beta('tod_mean', 0, None, None, 0)
tod_sd = Beta('tod_sd', 1, None, None, 0)
seas_mean = Beta('seas_mean', 0, None, None, 0)
seas_sd = Beta('seas_sd', 1, None, None, 0)

pf_rnd = pf_mean + pf_sd * Draws('pf_rnd', 'NORMAL')
cl_rnd = cl_mean + cl_sd * Draws('cl_rnd', 'NORMAL')
loc_rnd = loc_mean + loc_sd * Draws('loc_rnd', 'NORMAL')
wk_rnd = wk_mean + wk_sd * Draws('wk_rnd', 'NORMAL')
tod_rnd = tod_mean + tod_sd * Draws('tod_rnd', 'NORMAL')
seas_rnd = seas_mean + seas_sd * Draws('seas_rnd', 'NORMAL')

choice = Variable('choice')

V = {
    j: pf_rnd * X['pf'][j] + cl_rnd * X['cl'][j] + loc_rnd * X['loc'][j] + wk_rnd * X['wk'][j] + tod_rnd * X['tod'][j] + seas_rnd * X['seas'][j]
    for j in [1,2,3,4]
}

In [17]:
prob = models.logit(V, None, choice)
logprob = log(MonteCarlo(PanelLikelihoodTrajectory(prob)))

the_biogeme = bio.BIOGEME(
    database_train, logprob, number_of_draws=n_draws, seed=999, generate_yaml=False, generate_html=False
)
the_biogeme.model_name = 'model_b'
results = the_biogeme.estimate()
print(results)

The number of draws (600) is low. The results may not be meaningful. 


Results for model model_b
Nbr of parameters:		12
Sample size:			361
Observations:			3446
Excluded data:			0
Final log likelihood:		-3173.087
Akaike Information Criterion:	6370.174
Bayesian Information Criterion:	6416.84



In [23]:
pandas_parameters = get_pandas_estimated_parameters(estimation_results=results)
display(pandas_parameters)

,Name,Value,Robust std err.,Robust t-stat.,Robust p-value
0,pf_mean,-1.053524,0.057997,-18.165187,0.000000e+00
1,pf_sd,0.236364,0.023810,9.927219,0.000000e+00
2,cl_mean,-0.252155,0.028801,-8.754944,0.000000e+00
3,cl_sd,0.394744,0.028258,13.969337,0.000000e+00
4,loc_mean,2.395974,0.150277,15.943684,0.000000e+00
5,loc_sd,1.841314,0.166024,11.090650,0.000000e+00
6,wk_mean,1.584268,0.107127,14.788694,0.000000e+00
7,wk_sd,1.175335,0.119438,9.840517,0.000000e+00
8,tod_mean,-9.816967,0.496163,-19.785784,0.000000e+00
9,tod_sd,2.278416,0.233819,9.744351,0.000000e+00


In [18]:
P = {
    j: MonteCarlo(models.logit(V, None, j))
    for j in [1, 2, 3, 4]
}

simulate = {
    f'Prob_alt{j}': P[j]
    for j in [1, 2, 3, 4]
}

biogeme_sim = bio.BIOGEME(database_test, simulate)
biogeme_sim.model_name = 'per_choice_probs'

probs = biogeme_sim.simulate(results.get_beta_values())

# Test the results

Compute the brier score:

In [19]:
# Jaxlogit
y = np.reshape(y_test, (prob_jj.shape[0], -1))
print(sklearn.metrics.brier_score_loss(y, prob_jj))

0.6419672245619579


In [20]:
# xlogit
y = np.reshape(y_test, (prob_xx.shape[0], -1))
print(sklearn.metrics.brier_score_loss(y, prob_xx))

0.6414062150558543


In [21]:
# Biogeme
y = df_wide_test['choice']
print(sklearn.metrics.brier_score_loss(y, probs))

0.6425035924695395
